In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from animate_optimization import animate_opt
from IPython.display import Image, display
from noise import pnoise2

# Define the size of the terrain
width = 50
height = 50

# Initialize an array for the terrain data
terrain = np.zeros((height, width))

# Parameters for noise generation
scale = 25.0  # The scale of the noise
octaves = 3  # The number of levels of detail
persistence = 0.5  # The amplitude of each octave (reduces each octave)
lacunarity = 2.0  # The frequency of detail at each octave (increases each octave)

# Generate the noise-based terrain
for i in range(height):
    for j in range(width):
        terrain[i][j] = pnoise2(
            i / scale,
            j / scale,
            octaves=octaves,
            persistence=persistence,
            lacunarity=lacunarity,
            repeatx=width,
            repeaty=height,
            base=48,
        )

terrain += 1 + np.min(terrain)

# Plotting the generated terrain
plt.figure(figsize=(5, 5))
plt.imshow(terrain, cmap="hot")
plt.colorbar()
# plt.title("Generated Terrain Using Perlin Noise")

In [ ]:
import numpy as np
from skopt import gp_minimize
from skopt.space import Integer

# Assuming 'terrain' is your generated and normalized terrain array


# Define the objective function
def terrain_height(params):
    x, y = params
    return terrain[y, x]  # Access the height; note the order of indices


# Define the search space
space = [Integer(0, width - 1, name="x"), Integer(0, height - 1, name="y")]

# Run the optimization
result = gp_minimize(
    terrain_height,
    space,
    n_calls=20,
    verbose=False,
    noise=0.1**2,  # the noise level (optional)
    x0=[int(width / 2), int(height / 2)],
)

optimal_x, optimal_y = result.x
optimal_height = result.fun

print(f"Optimal location (x, y): ({optimal_x}, {optimal_y})")
print(f"Minimum terrain height at this location: {optimal_height}")

In [ ]:
plt.figure(figsize=(5, 5))
plt.imshow(terrain, cmap="hot", origin="lower")
plt.colorbar()
plt.scatter(optimal_x, optimal_y, color="g", s=100, marker="x")


# Extract the function values (the elevation at each point tested)
func_vals = result.func_vals

# Plot the progression
plt.figure(figsize=(4, 4))
plt.plot(func_vals, marker="o", linestyle="-", color="b")
plt.axhline(y=np.min(terrain), color="r", linestyle="--")
plt.title("Optimization Progression")
plt.xlabel("calls")
plt.ylabel("Objective Function Value")
plt.grid(True)


gif_filename = animate_opt(terrain, result)
display(Image(filename=gif_filename))